<a href="https://colab.research.google.com/github/Bot-Weeb/Coursera_Capstone1/blob/main/Coursera_Applied_data_Science_capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1 Preparing the Data frame
 NOTE: all libraries and packages are listed here for all 3 parts

In [50]:
import pandas as pd
from sklearn.cluster import KMeans
import folium
import numpy as np
import matplotlib as mpl
import matplotlib.colors as colors
import matplotlib.cm as cm
import requests
!pip install lxml

Packages have been imported and we shall scrape and count dataframes.

In [51]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df=pd.read_html(url)
print(len(df))

3


In [52]:
df=pd.DataFrame(df[0])
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


We now need to remove all rows where Borough is listed as not assigned. We choose df[0] simply because its the first dataframe.

In [53]:
noborough = df[df['Borough'] == 'Not assigned'].index
df.drop(noborough, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [54]:
df[df['Postal Code'].duplicated()]

,Postal Code,Borough,Neighbourhood


In [55]:
df[df['Neighbourhood']== 'Not assigned']

,Postal Code,Borough,Neighbourhood


The 2 cell blocks above ensures no duplication in postal codes, and makes neighbourhoods same as Borough. 

In [56]:
df = df.reset_index(drop=True)
df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [57]:
df.shape

(103, 3)

The dataframe is now complete and we know the shape.

# **Part 2 Get geographical coordinates**


We will use the pd.read_csv

In [58]:
!wget -O geocode.csv https://cocl.us/Geospatial_data
df_geoCode = pd.read_csv("geocode.csv")
df_geoCode.head()

--2021-01-30 17:49:55--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.63.96.176, 169.63.96.194
Connecting to cocl.us (cocl.us)|169.63.96.176|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-01-30 17:49:56--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2021-01-30 17:49:56--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Combine dataframe with part 1 dataframe

In [59]:
df_merged=pd.merge(df, df_geoCode, on='Postal Code')
df_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# **PART 3: Exploring and Clustering Neighbourhoods in Toronto**

The goal is to cluster neighbourhoods into categories based off of the frequency at which different types of venues appear. This will allow us to gain a better understanding at which areas favor which activities: as well as general popularity of different venues in Toronto.

In [60]:
df_toronto = df_merged[df_merged['Borough'].str.contains('Toronto')]
df_toronto.reset_index(drop=True, inplace=True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


The code above filters so that way we only recieve results pertaining to Toronto and then we will define the foursquare credentials

In [61]:
CLIENT_ID='F1OSDCXCWKIYRB2OTPFHGKCTHDLLJBM5S3ZJWZIKFNRDYJC4'
CLIENT_SECRET='F1SQMV5RJ3NEFQSHNVZJWW1BPLPYOS1PCHJPQ0YYZDFNG5AO'
VERSION= '20200623'

Next 2 cells will get category types  and explore neighbourhoods to get nearby venues.

In [62]:
 def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [63]:
LIMIT = 100
radius = 600

def getNearbyVenues(names, latitudes, longitudes, radius=600):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(name, lat, lng,
                             v['venue']['name'],
                             v['venue']['location']['lat'], 
                             v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue', 
                             'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)

Apply getNearbyVenues to all neighbourhoods

In [64]:
Toronto_Venues = getNearbyVenues(names=df_toronto['Neighbourhood'],longitudes=df_toronto['Longitude'],latitudes=df_toronto['Latitude'],)

In [65]:
Toronto_Venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


apply onehot encoding

In [66]:
toronto_onehot = pd.get_dummies(Toronto_Venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.rename(columns={'Neighbourhood':'Neighbourhood (category)'}, inplace=True)
toronto_onehot['Neighbourhood'] = Toronto_Venues['Neighbourhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,...,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Now we group by neighbourhood

In [67]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,...,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01087,0.0,0.0,0.01087,0.0,0.0,0.010870,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.01087,0.021739,0.000000,0.010870,0.0,0.01087,0.01087,0.0,0.021739,0.00,0.0,0.000000,0.01087,0.0,0.000000,0.0,0.000000,0.0,0.021739,...,0.000000,0.0,0.000000,0.0,0.021739,0.01087,0.0,0.000000,0.0,0.01087,0.0,0.0,0.000000,0.0,0.010870,0.0,0.0,0.01087,0.0,0.0,0.0,0.0,0.010870,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.021739,0.0,0.000000,0.0,0.000000,0.0,0.010870
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.025641,0.0,0.0,0.00,0.0,0.0,0.00000,0.025641,0.000000,0.025641,0.0,0.00000,0.00000,0.0,0.000000,0.00,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.051282,...,0.025641,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.025641,0.0,0.00000,0.0,0.0,0.025641,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.025641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.025641
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.05,0.0,0.0,0.00000,0.050000,0.000000,0.000000,0.0,0.00000,0.00000,0.0,0.000000,0.05,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.05000,0.0,0.0,0.000000,0.0,0.050000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.055556,0.055556,0.055556,0.111111,0.055556,0.111111,0.00000,0.0,0.0,0.00000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.0,0.00000,0.000000,0.000000,0.055556,0.0,0.00000,0.00000,0.0,0.000000,0.00,0.0,0.000000,0.00000,0.0,0.111111,0.0,0.055556,0.0,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.00000,0.0,0.0,0.010417,0.010417,0.000000,0.0,0.0,0.00,0.0,0.0,0.00000,0.000000,0.010417,0.010417,0.0,0.00000,0.00000,0.0,0.000000,0.00,0.0,0.010417,0.00000,0.0,0.000000,0.0,0.000000,0.0,0.010417,...,0.000000,0.0,0.010417,0.0,0.000000,0.00000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.020833,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.020833,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010417,0.0,0.010417,0.0,0.010417,0.0,0.010417


In [68]:
num_top_venues = 5
for hood in toronto_grouped['Neighbourhood']:
    print("---"+hood+"---")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---Berczy Park---
                venue  freq
0         Coffee Shop  0.09
1  Seafood Restaurant  0.04
2        Cocktail Bar  0.03
3              Lounge  0.03
4               Hotel  0.03


---Brockton, Parkdale Village, Exhibition Place---
            venue  freq
0     Coffee Shop  0.08
1            Café  0.08
2  Breakfast Spot  0.05
3      Restaurant  0.05
4  Sandwich Place  0.05


---Business reply mail Processing Centre, South Central Letter Processing Plant Toronto---
           venue  freq
0    Pizza Place  0.05
1  Auto Workshop  0.05
2        Brewery  0.05
3        Butcher  0.05
4     Steakhouse  0.05


---CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport---
              venue  freq
0    Airport Lounge  0.11
1  Airport Terminal  0.11
2       Coffee Shop  0.11
3     Boat or Ferry  0.11
4           Airport  0.06


---Central Bay Street---
             venue  freq
0      Coffee Shop  0.18
1             Café  0.08
2  Bubble Tea

Show some of the most common venues

In [69]:
def return_most_common_venues(row, num_top_venues):
  row_categories = row.iloc[1:]
  row_categories_sorted = row_categories.sort_values(ascending=False)
  return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
neighbourhood_venues_sorted=pd.DataFrame(columns=columns)
neighbourhood_venues_sorted['Neighbourhood']= toronto_grouped['Neighbourhood']
for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhood_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind,:], num_top_venues)
neighbourhood_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Restaurant,Café,Lounge,Japanese Restaurant,Hotel,Cocktail Bar,Breakfast Spot,Bakery
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Sandwich Place,Breakfast Spot,Gift Shop,Restaurant,Supermarket,Climbing Gym,Burrito Place,Department Store
2,"Business reply mail Processing Centre, South C...",Pizza Place,Bakery,Steakhouse,Beer Store,Brewery,Skate Park,Burrito Place,Butcher,Sandwich Place,Restaurant
3,"CN Tower, King and Spadina, Railway Lands, Har...",Boat or Ferry,Airport Lounge,Coffee Shop,Airport Terminal,Airport,Rental Car Location,Sculpture Garden,Plane,Harbor / Marina,Boutique
4,Central Bay Street,Coffee Shop,Café,Bubble Tea Shop,Italian Restaurant,Sandwich Place,Burger Joint,French Restaurant,Sushi Restaurant,Thai Restaurant,Salad Place


Utilize k-means clustering

In [70]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood',1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [73]:
neighbourhood_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df_toronto
toronto_merged = toronto_merged.join(neighbourhood_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.head()

ValueError: ignored

In [ ]:
toronto_coordinates =[43.6532, -79.3832]
map_clusters = folium.Map(location=toronto_coordinates, zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'],
                                  toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True,
                        fill_color=rainbow[cluster-1], fill_opacity=0.6).add_to(map_clusters)
map_clusters

In [ ]:
print('cluster 0')
toronto_merged.loc[toronto_merged['Cluster Labels']== 0, toronto_merged.columns[[2]+ list(range(6,toronto_merged.shape[1]))]]

In [ ]:
print('cluster 1')
toronto_merged.loc[toronto_merged['Cluster Labels']== 1, toronto_merged.columns[[2]+ list(range(6,toronto_merged.shape[1]))]] 

In [ ]:
print('cluster 2')
toronto_merged.loc[toronto_merged['Cluster Labels']== 2, toronto_merged.columns[[2]+ list(range(6,toronto_merged.shape[1]))]]

In [ ]:
print('cluster 3')
toronto_merged.loc[toronto_merged['Cluster Labels']== 3, toronto_merged.columns[[2]+ list(range(6,toronto_merged.shape[1]))]]

In [ ]:
print('cluster 4')
toronto_merged.loc[toronto_merged['Cluster Labels']== 4, toronto_merged.columns[[2]+ list(range(6,toronto_merged.shape[1]))]]